In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_finance import candlestick_ohlc
import math
import time
from datetime import datetime
import openpyxl
from itertools import groupby
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)


In [3]:
class account_balance:
    def __init__(self, btc, usd):
        self.btc = btc
        self.usd = usd

    def long_or_buy_btc(self, btc_earn, usd_spend):
        self.btc += btc_earn
        self.usd -= usd_spend

    def shout_or_sell_btc(self, btc_spend, usd_earn):
        self.btc -= btc_spend
        self.usd += usd_earn


In [4]:
class trade_strategy:
    def __init__(self,data):
        self.data = data
        self.previous_action = 0
        self.current_situation = 0
    
    def print_data(self, start, end):
        print(self.data.iloc[start:end,])

    def output_result(self):
        self.data.to_excel("Trading_Results.xlsx", sheet_name='Results')
        self.data.loc[(self.data["Action"] == "Buy")|(self.data["Action"] == "Sell")|(self.data["Action"] == "Long")|(self.data["Action"] == "Short")].to_csv("Each_Action.csv", sep = ',', encoding='utf-8')
        return
    
    def output_overview(self):
        # Part 1
        overview_array = []
        overview_array.append(["Action", "Win/Lose", "Times", "Average rate of return"])
        # Buy and win
        buy_win_times = len(self.data.loc[(self.data["Action"] == "Buy") & (self.data["Win/Lose"] == "Win")])
        buy_win_average = self.data["Rate of return"].loc[(self.data["Action"] == "Buy") & (self.data["Win/Lose"] == "Win")].mean()
        overview_array.append(["Buy", "Win", buy_win_times, buy_win_average])
        # Buy and lose
        buy_lose_times = len(self.data.loc[(self.data["Action"] == "Buy") & (self.data["Win/Lose"] == "Lose")])
        buy_lose_average = self.data["Rate of return"].loc[(self.data["Action"] == "Buy") & (self.data["Win/Lose"] == "Lose")].mean()
        overview_array.append(["Buy", "Lose", buy_lose_times, buy_lose_average])
        # Sell and win
        sell_win_times = len(self.data.loc[(self.data["Action"] == "Sell") & (self.data["Win/Lose"] == "Win")])
        sell_win_average = self.data["Rate of return"].loc[(self.data["Action"] == "Sell") & (self.data["Win/Lose"] == "Win")].mean()
        overview_array.append(["Sell", "Win", sell_win_times, sell_win_average])
        # Sell and lose
        sell_lose_times = len(self.data.loc[(self.data["Action"] == "Sell") & (self.data["Win/Lose"] == "Lose")])
        sell_lose_average = self.data["Rate of return"].loc[(self.data["Action"] == "Sell") & (self.data["Win/Lose"] == "Lose")].mean()
        overview_array.append(["Sell", "Lose", sell_lose_times, sell_lose_average])
        # Print array
        overview_header = overview_array.pop(0)
        overview_array = pd.DataFrame(overview_array, columns = overview_header)

        # Part 2
        total_trade = len(self.data.loc[(self.data["Action"] == "Buy") | (self.data["Action"] == "Sell")])
        total_win = len(self.data.loc[self.data["Win/Lose"] == "Win"])
        total_lose = len(self.data.loc[self.data["Win/Lose"] == "Lose"])
        total_gain  = sum(self.data["Gain/Loss"].loc[self.data["Gain/Loss"] > 0 ])
        total_loss = sum(self.data["Gain/Loss"].loc[self.data["Gain/Loss"] < 0 ])
        if total_win == 0 or total_loss == 0 or total_lose == 0:
            profit_factor = "None"
        else:
            profit_factor = abs((total_gain/total_win)/(total_loss/total_lose))
        if total_trade == 0:
            average_win_rate = "None"
        else:
            average_win_rate = total_win/total_trade
        
        # Part 3
        max_gain = self.data["Gain/Loss"].max()
        max_loss = self.data["Gain/Loss"].min()  
        max_gain_rate = self.data["Rate of return"].max()
        max_loss_rate= self.data["Rate of return"].min()
        win_lose_streak = self.data["Win/Lose"].loc[(self.data["Win/Lose"] == "Win")|(self.data["Win/Lose"] == "Lose")]
        # Calculate max streak
        groups = groupby(win_lose_streak)
        result = [[label, sum(1 for _ in group)] for label, group in groups]
        result = pd.DataFrame(result, columns=["Win/Lose", "Streak"])
        max_gains = result["Streak"].loc[result["Win/Lose"] == "Win"].max()
        max_losses = result["Streak"].loc[result["Win/Lose"] == "Lose"].max()
    
        # Output txt
        with open("Output.txt", "w") as text_file:
            # Part 1 output
            print(overview_array, file=text_file)
            print("=========================================", file=text_file)
            # Part 2 output
            print("總出手次數: {}" .format(total_trade), file=text_file)
            print("總獲利次數: {}" .format(total_win), file=text_file)
            print("總虧損次數: {}" .format(total_lose), file=text_file)
            print("總淨利: {}" .format(total_gain+total_loss), file=text_file)
            print("總獲利: {}" .format(total_gain), file=text_file)
            print("總虧損: {}" .format(total_loss), file=text_file)
            print("獲利因子(平均每筆獲利/平均每筆虧損): {}" .format(profit_factor), file=text_file)
            print("平均勝率: {}" .format(average_win_rate), file=text_file)
            print("=========================================", file=text_file)
            # Part 3 output
            print("歷史最大獲利: {}" .format(max_gain), file=text_file)
            print("歷史最大虧損: {}" .format(max_loss), file=text_file)
            print("歷史最大獲利率: {}" .format(max_gain_rate), file=text_file)
            print("歷史最大虧損率: {}" .format(max_loss_rate), file=text_file)
            print("最大連續獲利次數: {}" .format(max_gains), file=text_file)
            print("最大連續虧損次數: {}" .format(max_losses), file=text_file)  

    def apply_trading_strategy(self):
        self.data["Situation"] = 0
        self.data["Action"] = None
        self.data["Long"] = None
        self.data["Short"] = None
        self.data["Buy"] = None
        self.data["Sell"] = None
        self.data["Price"] = None
        self.data["Win/Lose"] = None
        self.data["Gain/Loss"] = None
        self.data["Rate of return"] = None

        first_action = True
        for i in range(len(self.data)):
            if first_action:
                #long or short
                if self.should_long(i):
                    self.trade_long(i)
                elif self.should_short(i):
                    self.trade_short(i)

                first_action = False
            elif self.do_next_trade(i,3):
                if self.current_situation > 0:
                    #long or sell
                    if self.should_sell(i):
                        self.trade_sell(i)
                elif self.current_situation < 0:
                    #short or buy
                    if self.should_buy(i):
                        self.trade_buy(i)
                else:
                    #long or short
                    if self.should_long(i):
                        self.trade_long(i)
                    elif self.should_short(i):
                        self.trade_short(i)
    
    def do_next_trade(self, i, period):
        if i >= self.previous_action + period:
            return True
        else:
            return False 

    def should_long(self, i):
        if self.data["SMA_20"].iloc[i] > self.data["SMA_60"].iloc[i]:
            return True
        else:
            return False
    
    def should_short(self, i):
        if self.data["Parabolic SAR"].iloc[i] > self.data["Low"].iloc[i]:
            return True
        else:
            return False

    def should_buy(self, i):
        if self.data["Parabolic SAR"].iloc[i] < self.data["High"].iloc[i]:
            return True
        else:
            return False

    def should_sell(self, i):
        if self.data["SMA_20"].iloc[i] < self.data["SMA_60"].iloc[i]:
            return True
        else:
            return False

    def trade_long(self, i):
        self.current_situation += 1
        self.data["Situation"].iloc[i] = self.current_situation
        self.data["Action"].iloc[i] = "Long"
        self.data["Long"].iloc[i] = 1
        self.data["Price"].iloc[i] = self.data["Close"].iloc[i]
        self.previous_action = i

    def trade_short(self, i):
        self.current_situation -= 1
        self.data["Situation"].iloc[i] = self.current_situation
        self.data["Action"].iloc[i] = "Short"
        self.data["Short"].iloc[i] = 1
        self.data["Price"].iloc[i] = self.data["Close"].iloc[i]
        self.previous_action = i 

    def trade_buy(self, i):
        self.current_situation += 1
        self.data["Situation"].iloc[i] = self.current_situation
        self.data["Action"].iloc[i] = "Buy"
        self.data["Buy"].iloc[i] = 1
        self.data["Price"].iloc[i] = self.data["Close"].iloc[i]
        previous_price = self.data["Close"].iloc[self.previous_action]
        current_price = self.data["Close"].iloc[i]

        if previous_price > current_price:
            self.data["Win/Lose"].iloc[i] = "Win"
            self.data["Gain/Loss"].iloc[i] = abs(current_price - previous_price)
            self.data["Rate of return"].iloc[i] = abs((current_price - previous_price)/previous_price)
        elif previous_price < current_price:
            self.data["Win/Lose"].iloc[i] = "Lose"
            self.data["Gain/Loss"].iloc[i] = -abs(current_price - previous_price)
            self.data["Rate of return"].iloc[i] = -abs((current_price - previous_price)/previous_price)
        else:
            self.data["Win/Lose"].iloc[i] = "Fair"
            self.data["Gain/Loss"].iloc[i] = 0
            self.data["Rate of return"].iloc[i] = 0
        
        self.previous_action = i
    
    def trade_sell(self, i):
        self.current_situation -= 1
        self.data["Situation"].iloc[i] = self.current_situation
        self.data["Action"].iloc[i] = "Sell"
        self.data["Sell"].iloc[i] = 1
        self.data["Price"].iloc[i] = self.data["Close"].iloc[i]
        previous_price = self.data["Close"].iloc[self.previous_action]
        current_price = self.data["Close"].iloc[i]

        if previous_price < current_price:
            self.data["Win/Lose"].iloc[i] = "Win"
            self.data["Gain/Loss"].iloc[i] = abs(current_price - previous_price)
            self.data["Rate of return"].iloc[i] = abs((current_price - previous_price)/previous_price)
        elif previous_price > current_price:
            self.data["Win/Lose"].iloc[i] = "Lose"
            self.data["Gain/Loss"].iloc[i] = -abs(current_price - previous_price)
            self.data["Rate of return"].iloc[i] = -abs((current_price - previous_price)/previous_price)
        else:
            self.data["Win/Lose"].iloc[i] = "Fair"
            self.data["Gain/Loss"].iloc[i] = 0
            self.data["Rate of return"].iloc[i] = 0

        self.previous_action = i

    def add_result(self, new_data):
        data.append(new_data)


In [5]:
class plot_data:
    def __init__(self, data):
        self.data = data
    
    def print_data(self, start, end):
        print(self.data.iloc[start:end,])
    
    def candlestick(self, time, open, high, low, close, volume):
        ohlc = self.data[[time, open, high, low, close, volume]]
        fig, ax = plt.subplots(figsize = (10,5))
                
        # plot candlestick
        candlestick_ohlc(ax, ohlc[[time, open, high, low, close]].values, width=1, colorup="g", colordown="r")
        ax.yaxis.set_label_position("left")
        ax.set_ylabel("Price", size=20)
        
        # shift y-limits of the candlestick plot so that there is space at the bottom for the volume bar chart        pad = 0.25
        pad = 0.75
        yl = ax.get_ylim()
        ax.set_ylim(yl[0] - (yl[1] - yl[0])*pad, yl[1])

        # create the second axis for the volume bar-plot
        ax2 = ax.twinx()  

        # set the position of ax2 so that it is short (y2=0.32) but otherwise the same size as ax
        # ax2.set_position(matplotlib.transforms.Bbox([[0.125,0.1],[0.9,0.32]]))

        # make bar plots and color differently depending on up/down for the day
        ohlc_positive = ohlc.loc[ohlc[open] - ohlc[close] <= 0]
        ohlc_negative = ohlc.loc[ohlc[open] - ohlc[close] > 0]

        # plot bar
        ax2.bar(ohlc_positive[time],ohlc_positive[volume],color='red',width=1,align='center')
        ax2.bar(ohlc_negative[time],ohlc_negative[volume],color='green',width=1,align='center')

        #scale the x-axis tight
        ax2.set_xlim(min(ohlc[time]),max(ohlc[time]))
        ax2.yaxis.set_label_position("right")
        ax2.set_ylabel('Volume', size=20)

        # format the x-ticks with a human-readable date. 
        ax.set_xticklabels(self.data["Datetime"],rotation=45, horizontalalignment='right')

        # plt.ioff()
        plt.show()

    def plot_all(self):
        INCREASING_COLOR = '#17BECF'
        DECREASING_COLOR = '#7F7F7F'
        
        data = [ dict(
        type = 'candlestick',
        open = self.data["Open"],
        high = self.data["High"],
        low = self.data["Low"],
        close = self.data["Close"],
        x = self.data["Datetime"],
        yaxis = 'y2',
        name = 'GS',
        increasing = dict( line = dict( color = INCREASING_COLOR ) ),
        decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
    ) ]

        layout=dict()

        fig = dict( data=data, layout=layout )

        fig['layout'] = dict()
        fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
        fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
        fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
        fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
        fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
        fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

        rangeselector=dict(
            visibe = True,
            x = 0, y = 0.9,
            bgcolor = 'rgba(150, 200, 250, 0.4)',
            font = dict( size = 13 ),
            buttons=list([
                dict(count=1,
                    label='reset',
                    step='all'),
                dict(count=1,
                    label='1yr',
                    step='year',
                    stepmode='backward'),
                dict(count=3,
                    label='3 mo',
                    step='month',
                    stepmode='backward'),
                dict(count=1,
                    label='1 mo',
                    step='month',
                    stepmode='backward'),
                dict(step='all')
            ]))
            
        fig['layout']['xaxis']['rangeselector'] = rangeselector

        fig['data'].append( dict( x=self.data["Datetime"], y=self.data["SMA_20"], type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

        colors = []

        for i in range(len(self.data["Close"])):
            if i != 0:
                if self.data["Close"].iloc[i] > self.data["Close"].iloc[i-1]:
                    colors.append(INCREASING_COLOR)
                else:
                    colors.append(DECREASING_COLOR)
            else:
                colors.append(DECREASING_COLOR)

        fig['data'].append( dict( x=self.data["Datetime"], y=self.data["Volume"],                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

        py.iplot( fig, filename = 'candlestick', validate = False )



In [6]:
class technical_analysis:
    def __init__(self, data):
        self.data = data
        self.ohlc_dict = {                                                                                                             
        'Timestamp':'last',
        'Open':'first',                                                                                                    
        'High':'max',                                                                                                       
        'Low':'min',                                                                                                        
        'Close': 'last',                                                                                                    
        'Volume': 'sum'
        }

    def print_data(self, start, end):
        print(self.data.iloc[start:end,])

    def add_real_datetime(self, time):
        self.data["Datetime"] = None
        self.data["Datetime"] = pd.to_datetime(self.data[time], unit='ms')

    def to_5min(self):
        self.data = self.data.set_index("Datetime")
        self.data = self.data.resample('5T', closed='right', label='right').apply(self.ohlc_dict)
        self.data = self.data.reset_index()

    def to_15min(self):
        self.data = self.data.set_index("Datetime")
        self.data = self.data.resample('15T', closed='right', label='right').apply(self.ohlc_dict)
        self.data = self.data.reset_index()

    def to_60min(self):
        self.data = self.data.set_index("Datetime")
        self.data = self.data.resample('60T', closed='right', label='right').apply(self.ohlc_dict)
        self.data = self.data.reset_index()

    def add_sma(self, close, period):
        sma_colname = "SMA_" + str(period)
        self.data[sma_colname] = None
        self.data[sma_colname] = self.data[close].rolling(period).mean()

    def add_parabolic_sar(self, time , high, low, close):
        iaf = 0.02
        maxaf = 0.2
        high = list(self.data[high])
        low = list(self.data[low])
        close = list(self.data[close])
        length = len(self.data)
        psar = close[0:len(close)]
        psarbull = [None] * length
        psarbear = [None] * length
        bull = True
        af = iaf
        ep = low[0]
        hp = high[0]
        lp = low[0]

        for i in range(2,length):
            if bull:
                psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
            else:
                psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
            
            reverse = False
            
            if bull:
                if low[i] < psar[i]:
                    bull = False
                    reverse = True
                    psar[i] = hp
                    lp = low[i]
                    af = iaf
            else:
                if high[i] > psar[i]:
                    bull = True
                    reverse = True
                    psar[i] = lp
                    hp = high[i]
                    af = iaf
        
            if not reverse:
                if bull:
                    if high[i] > hp:
                        hp = high[i]
                        af = min(af + iaf, maxaf)
                    if low[i - 1] < psar[i]:
                        psar[i] = low[i - 1]
                    if low[i - 2] < psar[i]:
                        psar[i] = low[i - 2]
                else:
                    if low[i] < lp:
                        lp = low[i]
                        af = min(af + iaf, maxaf)
                    if high[i - 1] > psar[i]:
                        psar[i] = high[i - 1]
                    if high[i - 2] > psar[i]:
                        psar[i] = high[i - 2]
                        
            if bull:
                psarbull[i] = psar[i]
            else:
                psarbear[i] = psar[i]

        self.data["Parabolic SAR"] = None
        self.data["SAR Bear"] = None
        self.data["SAR Bull"] = None
        self.data["Parabolic SAR"] = psar
        self.data["SAR Bear"] = psarbear
        self.data["SAR Bull"] = psarbull

        return
    
    def add_average_true_range(self, high, low, close, period):
        self.data["True range"] = None
        for i in range(len(self.data)):
            if i == 0:
                self.data["True range"].iloc[i] = self.data[high].iloc[i] - self.data[low].iloc[i] 
            else:
                tr_1 = self.data[high].iloc[i] - self.data[low].iloc[i]
                tr_2 = abs(self.data[high].iloc[i] - self.data[close].iloc[i-1])
                tr_3 = abs(self.data[low].iloc[i] - self.data[close].iloc[i-1])
                self.data["True range"].iloc[i] = max(tr_1, tr_2, tr_3)
        
        self.data["Average true range"] = None
        current_atr = 0
        if len(self.data) < period:
            return 0
        else:
            for i in range(len(self.data)):
                if i >= period-1:
                    if current_atr == 0:
                        current_atr = sum(self.data["True range"].iloc[i-period+1:i+1]) / period
                        self.data["Average true range"].iloc[i] = current_atr
                    else:
                        current_atr = (current_atr*(period - 1) + self.data["True range"].iloc[i]) / period
                        self.data["Average true range"].iloc[i] = current_atr        
    

In [7]:
class candlestick_period_transformation:
    def __init__(self, data):
        self.data = data


In [8]:
data = 0
ta_test = 0
plot = 0
ts_test = 0

data = pd.read_csv("./BTC_USD_2018.csv", header = None, names = ["Timestamp", "Open", "Close", "High", "Low", "Volume"])

# Technical analysis
ta_test = technical_analysis(data.iloc[0:50000,])
ta_test.add_real_datetime("Timestamp")
ta_test.to_15min()
ta_test.add_parabolic_sar("Datetime", "High", "Low", "Close")
ta_test.add_average_true_range("High", "Low", "Close", 14)
ta_test.add_sma("Close", 20)
ta_test.add_sma("Close", 60)
# ta_test.print_data(0,10)

# Trading strategy
ts_test = trade_strategy(ta_test.data) 
# ts_test.print_data(0,10)
ts_test.apply_trading_strategy()
ts_test.output_result()
ts_test.output_overview()


plot = plot_data(ta_test.data)
# plot.candlestick("Timestamp", "Open", "High", "Low", "Close", "Volume")
# plot.print_data(0,10)
plot.plot_all()


Aw, snap! We didn't get a username with your request.

Don't have an account? https://plot.ly/api_signup

Questions? accounts@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.